In [ ]:
import io, os, re, logging
import requests, tqdm, tarfile, itertools, html, time
import numpy as np
import pandas as pd
import dask.dataframe as dd

import plotly.express as px
from dateparser import parse

pd.set_option('display.max_rows', 20)

# Load all public shot IDs by iterating through pages from database

In [33]:
# n_pages = 7321

# def get_page_json(page_num):
#     response = requests.get(f'https://visualizer.coffee/api/shots?page={page_num}',)
#     obj = response.json()
#     return obj["data"]

# for page_num in tqdm.tqdm(range(1, n_pages+1)):
#     data = get_page_json(page_num)
#     data_pages.extend(data)

# pages_df = pd.DataFrame.from_dict(data_pages).drop_duplicates()

# Multi-worker 
# npartitions = 24
# pages = pd.Series(range(1589, n_pages+1))

# rest_df = dd.from_pandas(pages, npartitions=npartitions).apply(lambda pages: get_page_json(page), meta=list).compute()

# pages_df.to_csv("./gdrive/MyDrive/Software_Projects/VisualizerCoffee/visualizercoffee_shot_ids.csv", index=False)

In [8]:
# Load list of shots
public_shots_file = "./visualizercoffee_shot_ids.csv"
pages_df = pd.read_csv(public_shots_file)

pages_df

,clock,id
0,1636526780,14899484-a5f8-4ccc-b372-79700c8150ae
1,1636526752,76f936ee-5147-40ac-986e-f0f1be5ac97e
2,1636526738,8853ae7c-5bf8-4194-93dd-462de63f1471
3,1636526681,a4d7a358-fc3f-451c-92f5-756adaa4a7dd
4,1636526665,3ebcfff9-fb6f-47f6-a3df-affe01d55150
...,...,...
72888,1593858840,7ecca37f-2dae-429e-9947-b3ee08793b78
72889,1593855731,7bf138f6-8fb9-41c9-a513-74874115684f
72890,1593752661,b857b000-2326-4a90-83cc-f128253959df
72891,1593689393,f5dd375e-95e0-473a-90f0-2dca37fe0d29


In [19]:
def get_shot_json(shot_id):
    response = requests.get(f'https://visualizer.coffee/api/shots/{shot_id}/download',)
    obj = response.json()
    return obj

In [20]:
def json_to_series(data):
    if "error" in data: return False
    
    shot_df = pd.DataFrame({k:v for k,v in data.items() if k not in ["data", "timeframe"]}, 
                           index=pd.Index([data["id"]], name="id"))
    
    shot_df = pd.DataFrame.from_dict({k:v for k,v in data.items() if k not in ["data", "timeframe"]}, orient="index")[0]

    timeframe = pd.Index(data["timeframe"]+[data["timeframe"][-1]], name="timeframe").astype(float)
    shot_df["timeframe"] = timeframe.tolist()

    series_df = pd.DataFrame(data["data"]).astype(float)
    
    for key, values in series_df.to_dict(orient='list').items():
        shot_df[key] = np.array(values)

    return shot_df

In [21]:
# shot_series = []

# for shot_id in tqdm.tqdm(pages_df["id"]):
#     data = get_shot_json(shot_id)
    
#     try:
#         shot_json = json_to_series(data)
#         if not isinstance(shot_json, pd.Series): 
#             continue
#         shot_series.append(shot_json)
        
#     except:
#         logging.info(f"error for shot {shot_id}")

        
# shots_df = pd.concat(shot_series, axis=1).T
# shots_df.to_parquet("./visualizercoffee_shot_series.parquet", index=False)

100%|██████████████████████████████████| 72893/72893 [11:51:41<00:00,  1.71it/s]


# Import raw downloaded data

In [2]:
shots_df = pd.read_parquet("../data/raw/visualizercoffee_shot_series.parquet",)

In [3]:
shots_df.head()

,id,profile_title,user_id,drink_tds,drink_ey,espresso_enjoyment,bean_weight,drink_weight,grinder_model,grinder_setting,...,espresso_resistance,espresso_flow_weight,espresso_state_change,espresso_pressure_goal,espresso_flow_weight_raw,espresso_temperature_mix,espresso_water_dispensed,espresso_temperature_goal,espresso_resistance_weight,espresso_temperature_basket
0,14899484-a5f8-4ccc-b372-79700c8150ae,TurboBloom,8d827d10-510f-4475-bd54-63c93b275c16,0,0,80.0,15.0,35.2,P64 MP,1.4,...,"[0.0, 0.01, 0.01, 0.01, 0.0, 0.0, 0.01, 0.01, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 10000000.0, 10000000.0, 10000000.0, 1000...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[86.0, 86.67, 86.31, 82.06, 77.66, 74.73, 73.1...","[0.0, 0.092681884765625, 0.20140380859375, 0.3...","[86.0, 86.0, 86.0, 86.0, 86.0, 86.0, 86.0, 86....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[86.0, 85.12, 85.27, 85.49, 85.38, 85.35, 85.3..."
1,76f936ee-5147-40ac-986e-f0f1be5ac97e,Best overall pressure profile,10f1e281-8fd0-4f43-94f6-d416713dabe0,0,0,0.0,18.0,36.1,bentwood,120,...,"[0.0, 0.04, 0.03, 0.03, 0.02, 0.02, 0.02, 0.03...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 10000000.0, 10000000.0, 10000000.0, 1000...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[88.0, 88.3, 85.79, 85.82, 86.44, 87.32, 88.0,...","[0.0, 0.09389038085937501, 0.18845825195312502...","[88.0, 88.0, 88.0, 88.0, 88.0, 88.0, 88.0, 88....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[88.0, 87.18, 87.39, 87.37, 87.31, 87.23, 87.0..."
2,8853ae7c-5bf8-4194-93dd-462de63f1471,Default,30b3dd10-aebb-4f17-ba74-a1efcf2d51ea,0,0,0.0,None,40.3,None,None,...,"[0.0, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 10000000.0, 10000000.0, 10000000.0, 1000...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[90.5, 89.87, 87.2, 87.69, 87.95, 87.98, 88.18...","[0.0, 0.10127563476562501, 0.20233764648437502...","[90.5, 90.5, 90.5, 90.5, 90.5, 90.5, 90.5, 90....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[90.5, 87.93, 88.21, 88.23, 88.16, 88.14, 88.1..."
3,a4d7a358-fc3f-451c-92f5-756adaa4a7dd,Rao Allongé,c8c4793c-6708-4e81-90e8-6d8a5b30ceee,0,0,0.0,None,124.5,Niche Zero,13.5,...,"[0.0, 0.01, 0.01, 0.01, 0.02, 0.02, 0.02, 0.02...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 10000000.0, 10000000.0, 10000000.0, 1000...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[92.0, 91.63, 88.9, 87.29, 88.18, 89.33, 90.63...","[0.0, 0.094384765625, 0.19187011718750002, 0.2...","[92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[92.0, 90.0, 90.41, 90.36, 90.35, 90.44, 90.4,..."
4,3ebcfff9-fb6f-47f6-a3df-affe01d55150,Londonium,db110ce3-77b7-4c92-9d18-1019c3841cc9,0,0,0.0,None,0.0,None,None,...,"[0.0, 0.08, 0.07, 0.06, 0.06, 0.05, 0.04, 0.04...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 10000000.0, -10000000.0, -10000000.0, -1...","[-1.0, 2.0, 2.0, 2.375, 2.625, 2.75, 2.8125, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[89.0, 89.64, 87.11, 87.89, 88.81, 89.41, 89.6...","[0.0, 0.09944458007812501, 0.21490478515625, 0...","[89.0, 89.0, 88.5, 88.5, 88.5, 88.5, 88.5, 88....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[89.0, 87.55, 87.81, 87.75, 87.76, 87.8, 87.74..."


# Clean data

## Datetime

In [4]:
shots_df["start_time"] = pd.DatetimeIndex(shots_df["start_time"])
shots_df["start_time"] = shots_df["start_time"].dt.tz_localize(None)

In [5]:
def parse_date(x):
    if not isinstance(x, str) or len(x) < 4:
        return None        
    try: 
        dt = parse(x)
        timestamp = pd.Timestamp(dt)
    except:
        return None
    
    return timestamp
shots_df["roast_date"] = shots_df["roast_date"].apply(parse_date)

/opt/anaconda3/lib/python3.7/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
/opt/anaconda3/lib/python3.7/site-packages/dateparser/utils/__init__.py:130: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = tz.localize(date_obj)


In [6]:
shots_df["bean_age"] = shots_df["start_time"] - shots_df["roast_date"]

idx_A = shots_df["bean_age"] < pd.Timedelta(10, unit="minutes")
shots_df["bean_age"][idx_A] = None

idx_B = shots_df["bean_age"] > pd.Timedelta(200, unit="days")
shots_df["bean_age"][idx_B] = None

shots_df["bean_age"] = shots_df["bean_age"] / pd.to_timedelta(1, unit='D')

shots_df.loc[idx_A | idx_B]["roast_date"] = None

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus


### Timeframe series data

In [37]:
# Only needed when timeframe not aligned

def shift_timeframe_step(time_index, values, id=None):
    time_index = np.roll(time_index, shift=-1)
    time_index[-1] = time_index[-2]
    
    if len(time_index) - len(values) == 1:
        time_index = time_index[:-1]
        
    if len(time_index) != len(values):
        time_index = time_index[:len(values)]
    
    return time_index

shots_df["timeframe"] = shots_df[["timeframe", "espresso_flow", "id"]].apply(
    lambda x: shift_timeframe_step(x.timeframe, x.espresso_flow, x.id), axis=1)

In [38]:
shots_df["shot_time"] = shots_df["timeframe"].apply(max)

## Numericals

In [39]:
shots_df["drink_tds"] = shots_df["drink_tds"].str.replace(r"[^0-9.]+", '').replace({"": None})
shots_df["drink_ey"] = shots_df["drink_ey"].str.replace(",", ".").str.replace(r"[^0-9.]+", '').replace({"": None, "20210113174207": None})

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [40]:
shots_df["bean_weight"] = shots_df["bean_weight"].str.replace(",", ".").str.extract(r"(\d*\.\d+|\d+)")[0]
shots_df["drink_weight"] = shots_df["drink_weight"].str.replace(",", ".").str.extract(r"(\d*\.\d+|\d+)")[0]

In [41]:
shots_df.loc[shots_df["espresso_enjoyment"] > 100, "espresso_enjoyment"] = None

## Caterogrical fields

In [42]:
shots_df["profile_title"] = shots_df["profile_title"].replace({"": None})

In [43]:
shots_df = shots_df.astype({
    "drink_tds": float,
    "drink_ey": float,
    "bean_weight": float,
    "drink_weight": float,
    "espresso_enjoyment": float,
})

In [44]:
shots_df.dtypes.to_dict()

{'id': dtype('O'),
 'profile_title': dtype('O'),
 'user_id': dtype('O'),
 'drink_tds': dtype('float64'),
 'drink_ey': dtype('float64'),
 'espresso_enjoyment': dtype('float64'),
 'bean_weight': dtype('float64'),
 'drink_weight': dtype('float64'),
 'grinder_model': dtype('O'),
 'grinder_setting': dtype('O'),
 'bean_brand': dtype('O'),
 'bean_type': dtype('O'),
 'roast_date': dtype('<M8[ns]'),
 'espresso_notes': dtype('O'),
 'roast_level': dtype('O'),
 'bean_notes': dtype('O'),
 'start_time': dtype('<M8[ns]'),
 'image_preview': dtype('O'),
 'profile_url': dtype('O'),
 'timeframe': dtype('O'),
 'espresso_flow': dtype('O'),
 'espresso_weight': dtype('O'),
 'espresso_pressure': dtype('O'),
 'espresso_flow_goal': dtype('O'),
 'espresso_resistance': dtype('O'),
 'espresso_flow_weight': dtype('O'),
 'espresso_state_change': dtype('O'),
 'espresso_pressure_goal': dtype('O'),
 'espresso_flow_weight_raw': dtype('O'),
 'espresso_temperature_mix': dtype('O'),
 'espresso_water_dispensed': dtype('O'),

# Saved cleaned file

In [45]:
shots_df.to_parquet("../data/raw/visualizercoffee_72877shots_2021-11-11_cleaned.parquet", index=False)

# Show shot data

In [43]:
shot_df["image_preview"].values

array(['https://visualizer-coffee.s3.eu-central-1.amazonaws.com/screenshots/14899484-a5f8-4ccc-b372-79700c8150ae.png'],
      dtype=object)